**Important prerequisites**: before runnig this code files *EventResults.json* and *EventPredictions.json* must be present in the same foder. These files are a snapshot of Cosmos DB tables referenced in this project and obtained through [Azure Cosmos DB Desktop Data Migration Tool](https://github.com/azurecosmosdb/data-migration-desktop-tool).

In [ ]:
from collections import defaultdict
from datetime import datetime
import json
import pandas as pd

Loading actual results and getting rid of surplus data. It's also important to have sorted distinct values.

In [ ]:
class EventResult:
    def __init__(self, event_id, winner, commence_time):
        self.event_id = event_id
        self.winner = winner
        self.commence_time = commence_time

file_results = open('EventResults.json')
data_results = json.load(file_results)
file_results.close()

results = set([EventResult(result['id'], result['Winner'], result['CommenceTime']) for result in data_results])
results = sorted(results, key=lambda result: result.commence_time, reverse=True)

Loading predictions and fetching the most recent one for each event. There is only one prediction per event already.

In [ ]:
class EventPrediction:
    def __init__(self, event_id, winner, commence_time):
        self.event_id = event_id
        self.winner = winner
        self.commence_time = commence_time

file_predictions = open('EventPredictions.json')
data_predictions = json.load(file_predictions)

Calculating winrate

In [ ]:
total = 0
win_rate = []
wins = 0

for result in results:
    match = next((x for x in data_predictions if x['id'] == result.event_id), None)
    
    if match is not None:
        if result.winner == match['Winner']:
            wins += 1
        total += 1
        win_rate.append(float(wins) / float(total))

Visualizing win rate

In [ ]:
df = pd.DataFrame({ "win_rate": win_rate }, index = range(1, len(win_rate) + 1))
plot = df.plot(title="win_rate")